In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None          # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True   # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-14B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Qwen2 patching. Transformers: 4.53.1.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,    # Supports any, but = 0 is optimized
    bias = "none",       # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.12 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


## Data Preprocessing

In [4]:
import os
import torch
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, LlamaForSequenceClassification
from trl import SFTConfig, SFTTrainer
import pandas as pd
import numpy as np
# from unsloth.chat_templates import get_chat_template

# bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_compute_dtype=torch.float32
# )
seed = 52
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 4
checkpoint_path = 'Qwen/Qwen2.5-14B-Instruct'
debug = False

prompt = '''Analyze the following exam question and its four answer options. Your task is to determine the single most likely correct answer based on logical reasoning, textual patterns, and semantic alignment between the question and options. Consider keyword matching, contextual coherence, and elimination of implausible choices.

Format:
Question: [Full question text]
A: [Option A]
B: [Option B]
C: [Option C]
D: [Option D]

Answer: [Your Choice]

Respond with ONLY the letter (A/B/C/D) of the most probable correct answer, without explanations or additional text.
        '''
train_df = pd.read_csv('/kaggle/input/qa-vec/train.csv') # Данные из extra_df, смердженные с помощью эмбеддингов E5
test_df = pd.read_csv('/kaggle/input/qa-vec/test.csv')

In [5]:
def preprocess_text(row):
    q = row['question_stem']
    choices_text = eval(row['choices'])['text']  # Assuming this evaluates to a list of 4 items
    return pd.Series([q] + choices_text)

choices = ['Q', 'A', 'B', 'C', 'D']
train_df[choices] = train_df.apply(preprocess_text, 1)
test_df[choices] = test_df.apply(preprocess_text, 1)

In [6]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['answerKey'], random_state=19)

In [7]:
from datasets import load_dataset, Dataset
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_sharegpt

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
user_prompt = """Question: {}
A: {}
B: {}
C: {}
D: {}
Answer: """

# def get_context(fact_list):
#     return "\n\n".join(eval(fact_list))

# train_df['facts_context'] = train_df['fact_list'].apply(get_context)
# test_df['facts_context'] = test_df['fact_list'].apply(get_context)
dataset = [
    [{"role": "system", "content": prompt},
    {"role": "user", "content": user_prompt.format(q, a, b, c, d)},
    {"role": "assistant", "content": answer}] for q, answer, a, b, c, d in train_df[['Q', 'answerKey', 'A', 'B', 'C', 'D']].values]

val_dataset = [
    [{"role": "system", "content": prompt},
    {"role": "user", "content": user_prompt.format(q, a, b, c, d)},
    {"role": "assistant", "content": answer}] for q, answer, a, b, c, d in val_df[['Q', 'answerKey', 'A', 'B', 'C', 'D']].values]

def formatting(dataset):
    texts = []
    for i in range(len(dataset)):
        texts.append(tokenizer.apply_chat_template(dataset[i], tokenize=False, add_generation_prompt=False))
    return Dataset.from_dict({'text': texts})

dataset = formatting(dataset)
val_dataset = formatting(val_dataset)

## Model Training

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 905,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, # Fixed major bug in latest Unsloth
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 30,
        num_train_epochs = 0.5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "paged_adamw_8bit", # Save more memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/4911 [00:00<?, ? examples/s]

In [9]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=4):   0%|          | 0/4911 [00:00<?, ? examples/s]

In [10]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>system\nAnalyze the following exam question and its four answer options. Your task is to determine the single most likely correct answer based on logical reasoning, textual patterns, and semantic alignment between the question and options. Consider keyword matching, contextual coherence, and elimination of implausible choices.\n\nFormat:\nQuestion: [Full question text]\nA: [Option A]\nB: [Option B]\nC: [Option C]\nD: [Option D]\n\nAnswer: [Your Choice]\n\nRespond with ONLY the letter (A/B/C/D) of the most probable correct answer, without explanations or additional text.\n        <|im_end|>\n<|im_start|>user\nQuestion: A frog eats\nA: flowers\nB: grains\nC: six legged creatures\nD: cheeseburgers\nAnswer: <|im_end|>\n<|im_start|>assistant\nC<|im_end|>\n'

In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla P100-PCIE-16GB. Max memory = 15.888 GB.
12.041 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,911 | Num Epochs = 1 | Total steps = 614
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 68,812,800 of 14,000,000,000 (0.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


In [13]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

8589.6062 seconds used for training.
143.16 minutes used for training.
Peak reserved memory = 12.041 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 75.787 %.
Peak reserved memory for training % of max memory = 0.0 %.


## Model Validation and Inferencing

In [14]:
from tqdm import tqdm

In [15]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

val_dataset = [
    [{"role": "system", "content": prompt},
    {"role": "user", "content": user_prompt.format(q, a, b, c, d)}] for q, answer, a, b, c, d in val_df[['Q', 'answerKey', 'A', 'B', 'C', 'D']].values]
preds = []
for i in tqdm(range(len(val_df))):
    messages = val_dataset[i]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 16, use_cache = True,
                             temperature = 0.1, min_p = 0.1)
    preds.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][-1])

  0%|          | 0/546 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Qwen2ForCausalLM has no `_prepare_4d_causal_attention_mask_with_cache_position` method defined in its base modeling class. Compiled forward passes will be sub-optimal. If you're writing code, see Llama for an example implementation. If you're a user, please report this issue on GitHub.
100%|██████████| 546/546 [11:27<00:00,  1.26s/it]


In [16]:
from sklearn.metrics import accuracy_score

print("0.5 epoch, 905 tokens:")
accuracy_score(val_df['answerKey'], preds)

0.5 epoch, 905 tokens:


0.9413919413919414

In [17]:
val_dataset = [
    [{"role": "system", "content": prompt},
    {"role": "user", "content": user_prompt.format(q, a, b, c, d)}] for q, a, b, c, d in test_df[['Q', 'A', 'B', 'C', 'D']].values]
preds = []
for i in tqdm(range(len(test_df))):
    messages = val_dataset[i]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = model.generate(input_ids = inputs, max_new_tokens = 16, use_cache = True,
                             temperature = 0.1, min_p = 0.1)
    preds.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][-1])

100%|██████████| 500/500 [10:27<00:00,  1.26s/it]


In [18]:
sub = pd.read_csv("/kaggle/input/qa-vec/sample_submission.csv")
sub['answerKey'] = preds
# sub.loc[sub['answerKey'] == 'e', 'answerKey'] = 'D'
sub.to_csv('qa_vec_sub_qwen-14b-instruct-0-5epoch.csv', index=False)

In [19]:
sub['answerKey'].value_counts()

answerKey
A    134
C    133
B    122
D    111
Name: count, dtype: int64